In [1]:
import selenium


In [24]:
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import pandas as pd
from random import randint
import time



path = "/home/ozodbek/Documents/ecb-speeches/src/chromedriver"
service = Service(path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.ecb.europa.eu/press/pr/date/html/index.en.html")

# Variables to store the results
data = []
visited_links = set()

no_new_links_count = 0
MAX_ATTEMPTS = 5

actions = ActionChains(driver)

while no_new_links_count < MAX_ATTEMPTS:
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(randint(2, 4))  # Pause for content to load

    press_releases = driver.find_elements(By.CSS_SELECTOR, "dd")
    
    new_links_found = False
    for pr in press_releases:
        try:
            date_element = pr.find_element(By.XPATH, "./preceding-sibling::dt[1]")
            date = date_element.find_element(By.CLASS_NAME, "date").text
            link_element = pr.find_element(By.CLASS_NAME, "title").find_element(By.TAG_NAME, "a")
            link = link_element.get_attribute("href")
            
            if link not in visited_links:
                visited_links.add(link)
                new_links_found = True

                # Open link in a new tab and switch to it
                actions.key_down(Keys.CONTROL).click(link_element).key_up(Keys.CONTROL).perform()
                driver.switch_to.window(driver.window_handles[1])
                
                # Get the content
                content_element = driver.find_element(By.CSS_SELECTOR, "#main-wrapper > main > div.section")
                content = content_element.text
                data.append({"date": date, "content": content})
                
                # Close the new tab and switch back to the main tab
                driver.close()
                driver.switch_to.window(driver.window_handles[0])

        except Exception as e:
            print(f"Error processing press release: {e}")

    if not new_links_found:
        no_new_links_count += 1
    else:
        no_new_links_count = 0  # Reset if new links found

driver.close()

# Convert data to DataFrame and save as CSV
df3 = pd.DataFrame(data)
# df.to_csv("press_releases.csv", index=False)
df3

Error processing press release: list index out of range
Error processing press release: list index out of range
Error processing press release: list index out of range
Error processing press release: Message: element not interactable: https://www.ecb.europa.eu/stats/ecb_surveys/survey_of_professional_forecasters/html/ecb.spf2023q3~7fb4e0b3a7.en.html has no size and location
  (Session info: chrome=115.0.5790.170)
Stacktrace:
#0 0x559b9f9194e3 <unknown>
#1 0x559b9f648c76 <unknown>
#2 0x559b9f64d53c <unknown>
#3 0x559b9f64e97e <unknown>
#4 0x559b9f64ea4c <unknown>
#5 0x559b9f689f96 <unknown>
#6 0x559b9f689607 <unknown>
#7 0x559b9f6c4a1f <unknown>
#8 0x559b9f6a4012 <unknown>
#9 0x559b9f6bc30e <unknown>
#10 0x559b9f6a3de3 <unknown>
#11 0x559b9f6792dd <unknown>
#12 0x559b9f67a34e <unknown>
#13 0x559b9f8d93e4 <unknown>
#14 0x559b9f8dd3d7 <unknown>
#15 0x559b9f8e7b20 <unknown>
#16 0x559b9f8de023 <unknown>
#17 0x559b9f8ac1aa <unknown>
#18 0x559b9f9026b8 <unknown>
#19 0x559b9f902847 <unknown>
#

,date,content
0,10 August 2023,10 August 2023\nEuropa Open Air 2023 celebrate...
1,5 July 2023,5 July 2023\nCompared with April 2023:\nconsum...
2,4 July 2023,4 July 2023\nCredit terms and conditions tight...
3,28 June 2023,28 June 2023\nProposed legislation establishes...
4,22 June 2023,22 June 2023\nThe aggregate of total assets of...
...,...,...
297,18 September 1998,The European Central Bank (ECB) will today pub...
298,18 September 1998,The Headquarters Agreement between the Governm...
299,12 September 1998,In accordance with the Resolution adopted by t...
300,12 September 1998,Given that the euro banknotes will be put into...


In [25]:
df3

,date,content
0,10 August 2023,10 August 2023\nEuropa Open Air 2023 celebrate...
1,5 July 2023,5 July 2023\nCompared with April 2023:\nconsum...
2,4 July 2023,4 July 2023\nCredit terms and conditions tight...
3,28 June 2023,28 June 2023\nProposed legislation establishes...
4,22 June 2023,22 June 2023\nThe aggregate of total assets of...
...,...,...
297,18 September 1998,The European Central Bank (ECB) will today pub...
298,18 September 1998,The Headquarters Agreement between the Governm...
299,12 September 1998,In accordance with the Resolution adopted by t...
300,12 September 1998,Given that the euro banknotes will be put into...


In [28]:
df3.to_csv('src/data/ecb_releases_302.csv', index=False, encoding='utf-8-sig')